In [37]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import json

country = 'fi'  # Let's start with Germany as an example
start_date = '2024-01-01'
end_date = '2024-01-01'

url = "https://api.energy-charts.info/public_power"
params = {
    "country": country,
    "start": f"{start_date}T00:00+01:00",
    "end": f"{end_date}T23:45+01:00"
}

print("\n1. Making API request...")
response = requests.get(url, params=params)
data = response.json() 

entries = data['production_types']
entries_dict = {entry['name']: entry['data'] for entry in entries}
fossil_gas_data = entries_dict.get("Fossil gas")

##### TO BE EDITED ######


1. Making API request...


In [44]:
sum(fossil_gas_data)/4

4411.175

In [ ]:

# Find and extract Fossil gas data
gas_data = None
for prod_type in data['production_types']:
    if prod_type['name'] == 'Fossil gas':
        gas_data = prod_type

if gas_data and gas_data['data']:  # Check if we have data
    # Create lists of timestamps and values, filtering out None values
    timestamps = []
    values = []
    for ts, val in zip(data['unix_seconds'], gas_data['data']):
        if val is not None:  # Only include non-None values
            timestamps.append(ts)
            values.append(val * 2)  # Multiply by 2
    
    # Create DataFrame with filtered data
    df = pd.DataFrame({
        'timestamp': [datetime.utcfromtimestamp(ts) for ts in timestamps],
        'demand': values
    })

    df['timestamp'] = df['timestamp'] + pd.DateOffset(hours=1)  # Added line to shift timestamps
    
    # Convert to daily values
    daily_df = pd.DataFrame({
        'country': country.upper(),
        'date': df.groupby(df['timestamp'].dt.date)['timestamp'].first(),
        'demand': df.groupby(df['timestamp'].dt.date)['demand'].sum(),
        'type': 'power'
    }).reset_index(drop=True)


In [ ]:
df = pd.DataFrame({
    'timestamp': [datetime.utcfromtimestamp(ts) for ts in timestamps],
    'demand': values
})

In [35]:
len(fossil_gas_data)

96

In [36]:
len(data['unix_seconds'])

96

In [18]:
len(data['unix_seconds'])

96

In [28]:
entries_dict.keys()

dict_keys(['Cross border electricity trading', 'Nuclear', 'Hydro Run-of-River', 'Biomass', 'Fossil hard coal', 'Fossil peat', 'Fossil oil', 'Fossil gas', 'Others', 'Other renewables', 'Waste', 'Wind onshore', 'Solar', 'Load', 'Residual load', 'Renewable share of generation', 'Renewable share of load'])

In [22]:
pd.DataFrame(entries)

,name,data
0,Cross border electricity trading,"[2271.7, 2271.7, 2271.7, 2271.7, 1977.7, 1977...."
1,Nuclear,"[4349.3, 4350.3, 4351.2, 4350.7, 4350.2, 4350...."
2,Hydro Run-of-River,"[1155.1, 1236.4, 1225.8, 1231.3, 1194.9, 1147...."
3,Biomass,"[833.1, 817.9, 802.3, 805.9, 802.1, 799.5, 796..."
4,Fossil hard coal,"[187.8, 188.4, 188.3, 187.7, 188.6, 190.2, 192..."
5,Fossil peat,"[482.9, 481.1, 485.2, 482.4, 478.7, 479.6, 478..."
6,Fossil oil,"[0.30000000000000004, 0.30000000000000004, 0.3..."
7,Fossil gas,"[181.7, 177.4, 175.3, 178.3, 178.2, 176.5, 171..."
8,Others,"[243.6, 244.2, 247.1, 244.7, 247.1, 247.1, 247..."
9,Other renewables,"[1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, ..."
